# Generate exercises using a large language model (LLM)
This is an implementation of the proposal https://miro.com/app/board/uXjVMuVH2kA=/ made by the author unit.

## LLM setup
We use OpenAI for now because we have not yet investigated other options. Might want to do that, though.

In [1]:
!pip install python-dotenv
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
#load_dotenv(find_dotenv())
load_dotenv("OPENAI.env") # read local .env file
openai.api_key = os.environ.get('OPENAI_API_KEY')

/Users/annas/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Prompt setup
First we define some enums as intended by the prototype:

In [6]:
from enum import Enum

class Difficulty(Enum):
    LOW: str = "leicht"
    MEDIUM: str = "moderat"
    HIGH: str = "knifflig"

class ExerciseCategory(Enum):
    SINGLE: str = "eine Einzelaufgabe"
    QUIZ: str = "ein Quiz"
    TEST: str = "einen Test"

class ExerciseType(Enum):
    MULTIPLE_CHOICE: str = "Multiple Choice"
    SINGLE_CHOICE: str = "Single Choice"
    SHORT_ANSWER: str = "Short Answer"
    TRUE_FALSE: str = "Wahr Falsch"
    MAPPING: str = "Zuordnung"
    FREE_TEXT: str = "Freitext"
    FACTUAL_TASK: str = "Sachaufgabe"
    SINGLE_WORD_SOLUTION: str = "Lösung mit 1 Wort"
    SINGLE_NUMBER_SOLUTION: str = "Lösung mit 1 Zahl"

Now we set the actual variables that the frontend would gather, feel free to change the values and experiment:

In [7]:
subject: str = "Mathe"
grade: int = 8
level: Difficulty= Difficulty.MEDIUM
topic: str = "Bruchrechnung"
goal: str = "Die Schüler*innen können Brüche erweitern und kürzen."
category: ExerciseCategory = ExerciseCategory.QUIZ
exercise_types: list[ExerciseType] = [ExerciseType.MULTIPLE_CHOICE, ExerciseType.SINGLE_CHOICE, ExerciseType.SHORT_ANSWER] 
number_exercises: int = 10
info: str = """Erstelle erst die Aufgabenstellung. Erstelle dann für die Multiple Choice Aufgaben mindestens zwei richtige und mehrere falsche Antworten, für die Single Choice Aufgabe genau eine richtige Antwort und mehrere falsche Antworten. Bei Short Answer Aufgaben gibt der Schüler die Antwort selbst an. 

Das JSON-Format für jede Aufgabe sieht so aus: { "type": <"multiple_choice" or "single_choice" or "short_answer" depending on the exercise>, "question": <text string>, "options": <list of possible answers>, "correct_options": <list of indices indicating which answer from "options" is correct> } 
Gib nur eine Liste von JSON-Objekten mit diesem Format zurück: [{}].
Formatiere alle Mathe-Symbole als Latex. 

Beachte: In der Ausgabe soll kein Plain Text, sondern nur JSON stehen!"""

We use the framework langchain because it is convenient, though tasks of this level of complexity could also be done without.

In [7]:
!pip install langchain

452.04s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/5c/c2/66a16f85f5fc275ba3436a7862d7d89f736682687e2c93359e8ab6541dae/langchain-0.0.283-py3-none-any.whl.metadata
  Obtaining dependency information for SQLAlchemy<3,>=1.4 from https://files.pythonhosted.org/packages/4b/dd/8fe0fc21fc4338e7479f1b254a67b5515bd31b85c28925045bc8b0d5a1c3/SQLAlchemy-2.0.20-cp39-cp39-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for dataclasses-json<0.6.0,>=0.5.7 from https://files.pythonhosted.org/packages/97/5f/e7cc90f36152810cab08b6c9c1125e8bcb9d76f8b3018d101b5f877b386c/dataclasses_json-0.5.14-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.1.0,>=0.0.21 from https://files.pythonhosted.org/packages/95/3f/2fdeb0af80d210bce2b0bf2da565f7fb9206036ee0adb358759bb994e28d/langsmith-0.0.33-py3-none-any.whl.metadata
  Obtaining dependency information f

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template_string = """Erstelle {number_exercises} verschiedene Aufgaben für ein für {category} \
in {subject} \
in der Jahrgangsstufe {grade}. \
Das Quiz wird im Rahmen des Unterrichts an einer Mittelschule eingesetzt. \
Der Schwierigkeitsgrad soll {level} sein. \
Das Ziel der Aufgaben ist: {goal}. \
Die Schülerinnen kennen die Grundlagen der Bruchrechnung. Folgende Aufgabentypen sollen hierbei enthalten sein: {exercise_types}. \
{info}
"""

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                """Du bist eine Software, die Aufgaben in einem JSON-Format für \
Schüler für den Mathematikunterricht erstellt. Die Aufgaben sollen \
für ein Quiz genutzt werden. Das Ziel der Aufgaben ist, dass die \
Schüler ihre Kenntnisse in Mathematik üben und verbessern können. \
Du sollst keinen Text, sondern nur ein JSON-Objekt zurückgeben."""
            )
        ),
        HumanMessagePromptTemplate.from_template(template_string),
    ]
)

chat = ChatOpenAI(openai_api_key=openai.api_key, temperature=0.0)

prompt_to_generate_exercises = template.format_messages(subject=subject, 
                                                               grade=grade, 
                                                               level=level.value, 
                                                               topic=topic, 
                                                               goal=goal, 
                                                               category=category.value, 
                                                               exercise_types=', '.join([item.value for item in exercise_types]), 
                                                               number_exercises=number_exercises, 
                                                               info=info
                                                              )
print(prompt_to_generate_exercises)

[SystemMessage(content='Du bist eine Software, die Aufgaben in einem JSON-Format für Schüler für den Mathematikunterricht erstellt. Die Aufgaben sollen für ein Quiz genutzt werden. Das Ziel der Aufgaben ist, dass die Schüler ihre Kenntnisse in Mathematik üben und verbessern können. Du sollst keinen Text, sondern nur ein JSON-Objekt zurückgeben.', additional_kwargs={}), HumanMessage(content='Erstelle 10 verschiedene Aufgaben für ein für ein Quiz in Mathe in der Jahrgangsstufe 8. Das Quiz wird im Rahmen des Unterrichts an einer Mittelschule eingesetzt. Der Schwierigkeitsgrad soll moderat sein. Das Ziel der Aufgaben ist: Die Schüler*innen können Brüche erweitern und kürzen.. Die Schülerinnen kennen die Grundlagen der Bruchrechnung. Folgende Aufgabentypen sollen hierbei enthalten sein: Multiple Choice, Single Choice, Short Answer. Erstelle erst die Aufgabenstellung. Erstelle dann für die Multiple Choice Aufgaben mindestens zwei richtige und mehrere falsche Antworten, für die Single Choice 

## Generate exercises
Now we see in what this prompt results:

In [17]:
generated_exercises = chat(prompt_to_generate_exercises)
print(generated_exercises.content)

[
  {
    "type": "multiple_choice",
    "question": "Welche der folgenden Optionen erweitert den Bruch $\\frac{3}{4}$ auf $\\frac{9}{12}$?",
    "options": [
      "$\\frac{1}{2}$",
      "$\\frac{2}{3}$",
      "$\\frac{3}{5}$",
      "$\\frac{4}{6}$"
    ],
    "correct_options": [1, 3]
  },
  {
    "type": "single_choice",
    "question": "Welche der folgenden Optionen kürzt den Bruch $\\frac{12}{16}$ auf $\\frac{3}{4}$?",
    "options": [
      "$\\frac{2}{3}$",
      "$\\frac{3}{4}$",
      "$\\frac{4}{5}$",
      "$\\frac{5}{6}$"
    ],
    "correct_options": [0]
  },
  {
    "type": "short_answer",
    "question": "Kürze den Bruch $\\frac{16}{24}$ auf den kleinstmöglichen Bruch."
  },
  {
    "type": "multiple_choice",
    "question": "Welche der folgenden Optionen erweitert den Bruch $\\frac{5}{8}$ auf $\\frac{15}{24}$?",
    "options": [
      "$\\frac{1}{3}$",
      "$\\frac{2}{4}$",
      "$\\frac{3}{5}$",
      "$\\frac{4}{6}$"
    ],
    "correct_options": [1, 3]
  },
  {